# PoP NBr12-14

In [ ]:
from WS_Mdl import utils as U
from WS_Mdl import geo as G
# import os
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
# from pathlib import Path

In [ ]:
import os
from datetime import datetime as DT
from WS_Mdl.geo import _HD_IDF_Mo_Avg_to_MBTIF_process_Mo 
N_cores = None
from concurrent.futures import ProcessPoolExecutor as PPE

In [ ]:
import importlib
importlib.reload(G)
importlib.reload(U)

# Options

In [ ]:
#l_MdlN = [f'NBr{i}' for i in range(12, 14+1)]

# Calculate Mo AVG HDs maps for each L of a Sim

In [ ]:
MdlN = 'NBr13'
L = None

In [ ]:
d_paths = U.get_MdlN_paths(MdlN)
path_PoP, path_HD = [ d_paths[v] for v in ['path_PoP', 'path_Out_HD'] ]

In [ ]:
DF = U.HD_Out_IDF_to_DF(path_HD) # Read the IDF files to a DataFrame
DF_grouped = DF.groupby(['year', 'month'])['path']

In [ ]:
DF.sort_values(['day', 'L'])

In [ ]:
path_Mo_AVG_Fo = os.path.join(path_PoP, f'Out/{MdlN}/HD_Mo_AVG') # path where Mo AVG files are stored
os.makedirs(path_Mo_AVG_Fo, exist_ok=True) # Create the directory if it doesn't exist

if N_cores is None:
    N_cores = max(os.cpu_count() - 2, 1) # Leave 2 cores free for other tasks. If there aren't enough cores available, set to 1.

start = DT.now() # Start time
with PPE(max_workers=N_cores) as E:
    futures = [E.submit(_HD_IDF_Mo_Avg_to_MBTIF_process_Mo, year, month, list(paths), MdlN, path_Mo_AVG_Fo, path_HD, crs)
                for (year, month), paths in DF_grouped]
    for f in futures:
        print('\t', f.result(), '- Elapsed time (from start):', DT.now() - start)

In [ ]:
for MdlN in l_MdlN:
    G.HD_IDF_Mo_Avg_to_MBTIF(MdlN)

# Calculate differences

# GXG

In [ ]:
for MdlN in l_MdlN:
    G.HD_IDF_GXG_to_MBTIF(MdlN)